In [1]:
import findspark
findspark.init() # this must be executed before the below import

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark import SparkFiles

In [ ]:
# spark = SparkSession.builder.master('local').appName('myAppName').getOrCreate()

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.csv(SparkFiles.get('/home/cloudray/Downloads/TPCH_12M_8Field.csv'), header=False, inferSchema=True)

In [5]:
subset_rows = df.filter(df._c2 < 2).collect()

In [6]:
subset_rdd = sc.parallelize(subset_rows)

In [7]:
subset_df = subset_rdd.toDF()

In [8]:
df.count()
subset_df.count()

571

In [ ]:
# write to parquet (HDFS)

In [12]:
df.write.parquet('hdfs://localhost:9000/user/cloudray/NORA/partition_0.parquet')

In [11]:
subset_df.write.parquet('hdfs://localhost:9000/user/cloudray/NORA/partition_1.parquet')

In [ ]:
# read from parquet (HDFS)

In [15]:
df2 = sqlContext.read.parquet('hdfs://localhost:9000/user/cloudray/NORA/partition_1.parquet')

In [16]:
print(df2)

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: double, _c5: double, _c6: double, _c7: double]


In [18]:
df2.head()

Row(_c0=21378.0, _c1=400000.0, _c2=1.0, _c3=3.0, _c4=47.0, _c5=51699.53, _c6=0.01, _c7=0.06)

In [20]:
df2.count()

571

In [24]:
# df2.filter(df2._c2 < 2).collect()

In [ ]:
# testing locading multiple parquet files at once

In [25]:
def get_parquet_file_paths(partition_ids):
    
    hdfs_path = 'hdfs://localhost:9000/user/cloudray/NORA/'
    result_paths = []
    
    for pid in partition_ids:
        partition_name = 'partition_' + str(pid)+'.parquet'
        path = hdfs_path + partition_name
        result_paths.append(path)
        
    return result_paths

In [26]:
paths = get_parquet_file_paths([0,1])

In [27]:
print(paths)

['hdfs://localhost:9000/user/cloudray/NORA/partition_0.parquet', 'hdfs://localhost:9000/user/cloudray/NORA/partition_1.parquet']


In [28]:
dfs = sqlContext.read.parquet(*paths)

In [29]:
print(dfs)

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: double, _c5: double, _c6: double, _c7: double]


In [30]:
dfs.count()

11998567

In [39]:
def try_pass_sqlcontext(context, df):
    df = context.read.parquet('hdfs://localhost:9000/user/cloudray/NORA/partition_1.parquet')
    return df

In [42]:
try_pass_sqlcontext(sqlContext, dfs)

In [43]:
dfs.count()

11998567